In [1]:
import numpy as np
import torch
torch.backends.cuda.enable_mem_efficient_sdp(False)
from transformers import LlamaForSequenceClassification
model = LlamaForSequenceClassification.from_pretrained('/kaggle/input/prm-code',\
                                                    num_labels=1,\
                                                    device_map="cuda",
                                                    torch_dtype="auto",
                                                    ignore_mismatched_sizes=True,
                                                    ).eval()
model.score.load_state_dict(torch.load('/kaggle/input/prm-code/model_score7_code.pth'))
model = model.model

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at /kaggle/input/prm-code and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<All keys matched successfully>

In [9]:
# Generate random integer inputs
input1 = torch.randint(0, 50000, (1, 600),device='cuda')  # Random sequence of length 5
input2 = torch.randint(0, 50000, (1, 200),device='cuda')  # Random sequence of length 5

# Run forward pass on input1 and save past_key_values
with torch.no_grad():
    outputs1 = model(input1, use_cache=True)
    past_key_values = outputs1.past_key_values

# Run forward pass on input2 with saved past_key_values
with torch.no_grad():
    outputs2 = model(input2, past_key_values=past_key_values, use_cache=True)

# Run forward pass on combined input1 and input2
with torch.no_grad():
    combined_input = torch.cat([input1, input2], dim=1)
    outputs_combined = model(combined_input)

# Extract the last hidden states
last_hidden_state_1_2 = outputs2.last_hidden_state
last_hidden_state_combined = outputs_combined.last_hidden_state

# Compare the outputs
comparison = torch.allclose(last_hidden_state_1_2, last_hidden_state_combined[:, -input2.size(1):])

print(f"Are the outputs the same? {comparison}")

Are the outputs the same? True


In [ ]:
input1 = input1.to('cpu')
combined_input = combined_input.to('cpu')

In [15]:
%%time
with torch.no_grad():
    input1 = input1.to('cuda')
    outputs2 = model(input2, past_key_values=past_key_values, use_cache=True)

CPU times: user 318 ms, sys: 278 ms, total: 596 ms
Wall time: 596 ms


In [16]:
%%time
with torch.no_grad():
    combined_input = combined_input.to('cuda')
    outputs_combined = model(combined_input)

CPU times: user 896 ms, sys: 781 ms, total: 1.68 s
Wall time: 1.68 s
